## First Attempt at building a deepsets architecture

In [9]:
import random

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from astropy.io import fits
import healpy as hp
from set_dataloader import CCD




#### Experimenting with input sizes

In [ ]:
# For DECAM, BASS, MzLS
with open('../../bricks_data/pixel2ccd_2048_non_inclusive.pickle', 'rb') as f:
    subpixel2ccd_dict = pickle.load(f)
    f.close()

In [ ]:
print(len(subpixel2ccd_dict))

In [ ]:
min = 1000
max = 0
aggregate = 0
index_max_sub = 0
lens = np.zeros(len(subpixel2ccd_dict))

for i,pix in enumerate(subpixel2ccd_dict.keys()):
    no_ccds = len(subpixel2ccd_dict[pix])
    lens[i] = no_ccds
    aggregate += no_ccds
    if no_ccds > max:
        max = no_ccds
        index_max_sub = pix
    if no_ccds < min:
        min = no_ccds

print("Mean CCD's per subpixel:", lens.mean())
print("Max CCD's per subpixel:", max)
print("Min CCD's per subpixel:", min)



In [ ]:
plt.hist(lens, bins=[0,30,50,100])

In [ ]:
with open('../../bricks_data/pixel2ccd_256_non_inclusive.pickle', 'rb') as f:
    pixel2ccd_dict = pickle.load(f)
    f.close()

In [ ]:
min = 1000
max = 0
aggregate = 0
index_max = 0
lens = np.zeros(len(pixel2ccd_dict))

for i,pix in enumerate(pixel2ccd_dict.keys()):
    no_ccds = len(pixel2ccd_dict[pix])
    lens[i] = no_ccds
    aggregate += no_ccds
    if no_ccds > max:
        max = no_ccds
        index_max = pix
    if no_ccds < min:
        min = no_ccds

print("Mean CCD's per pixel:", lens.mean())
print("Max CCD's per pixel:", max)
print("Min CCD's per pixel:", min)


In [ ]:
plt.hist(lens, bins=[0,30,50,100])

In [ ]:
ccd = CCD()

ra0, dec0, ra1, dec1, ra2, dec2, ra3, dec3 = ccd.get_boundaries()



### Plotting the Pixel - CCD Relationship for the most populated pixel

In [ ]:
#Validating everything went as intended visually

NSIDE = 256
pixel_number = index_max
# print(pixel2ccd_dict.keys())
print(len(pixel2ccd_dict.keys()))


ccd = pixel2ccd_dict[pixel_number]
print("Number of CCDs that are cutting pixel number ", pixel_number, ":", len(ccd))
for cc in ccd:
#coord = [[x0,y0], [x1,y1], [x2,y2], [x3,y3], [x0,y0]]
    xs = [ra0[cc],ra1[cc], ra2[cc], ra3[cc],ra0[cc] ]
    ys = [dec0[cc],dec1[cc], dec2[cc], dec3[cc],dec0[cc]]
    plt.plot(xs,ys)
#if i > 3:
    #break

pixel_boundary = hp.boundaries(nside=NSIDE, pix=pixel_number, step=1)
pixel_boundary_raDec = hp.vec2ang(pixel_boundary.transpose(),lonlat=True)
pixel_boundary_raDec = list(pixel_boundary_raDec)
pixel_boundary_raDec[0] = np.append(pixel_boundary_raDec[0],pixel_boundary_raDec[0][0])
pixel_boundary_raDec[1] = np.append(pixel_boundary_raDec[1],pixel_boundary_raDec[1][0])


plt.plot(pixel_boundary_raDec[0],pixel_boundary_raDec[1], c='black', label="Pixel Boundary")

plt.show()

In [ ]:
#Validating everything went as intended visually

NSIDE = 2048
pixel_number = index_max_sub
# print(pixel2ccd_dict.keys())
print(len(subpixel2ccd_dict.keys()))


ccd_sub = subpixel2ccd_dict[pixel_number]
print("Number of CCDs that are cutting pixel number ", pixel_number, ":", len(ccd))
for cc in ccd_sub:
#coord = [[x0,y0], [x1,y1], [x2,y2], [x3,y3], [x0,y0]]
    xs = [ra0[cc],ra1[cc], ra2[cc], ra3[cc],ra0[cc] ]
    ys = [dec0[cc],dec1[cc], dec2[cc], dec3[cc],dec0[cc]]
    plt.plot(xs,ys)
#if i > 3:
    #break

pixel_boundary = hp.boundaries(nside=NSIDE, pix=pixel_number, step=1)
pixel_boundary_raDec = hp.vec2ang(pixel_boundary.transpose(),lonlat=True)
pixel_boundary_raDec = list(pixel_boundary_raDec)
pixel_boundary_raDec[0] = np.append(pixel_boundary_raDec[0],pixel_boundary_raDec[0][0])
pixel_boundary_raDec[1] = np.append(pixel_boundary_raDec[1],pixel_boundary_raDec[1][0])


plt.plot(pixel_boundary_raDec[0],pixel_boundary_raDec[1], c='black', label="Pixel Boundary")

plt.show()

In [ ]:
x = [x for x in ccd_sub if x not in ccd]

print(len(x))

## Building a deepsets architecture

In [3]:
# Import NN Packages
import torch
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing, metrics
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt


print(torch.__version__)


1.9.0


## Writing Dataset Utility to pass data in the right format

#### Steps:

1. Build a NN sampling an equal number of CCDs per 256 pixel and pass through deep sets for regression

2. Adapt NN for variable sized inputs

3. Adapt NN to use 64 inputs of size 2048 to then predict density at 256


In [33]:
# noinspection PyAttributeOutsideInit
class SetSequence(Dataset):
    """Processes and Returns a Dataset of Variable Sized Input Sets of Dimensions
    N = Number Pixels of that are returned
    M = Max Size of each Individual Set of CCDs


    """

    def __init__(self, num_pixels = 10, max_ccds = 30, var_set_len = False):

        with open('../../bricks_data/pixel2ccd_256_non_inclusive.pickle', 'rb') as f:
            self.pixel2ccd_dict = pickle.load(f)
            f.close()


        self.ccd = CCD()
        self.num_features = self.ccd.num_features

        # Dimensions
        self.num_pixels = num_pixels
        self.max_ccds = max_ccds
        self.var_set_len = var_set_len

        df_raw = pd.read_csv('../../bricks_data/dataset_geometric.csv')
        # Randomly Sampling Pixel Indices from Dataframe
        pixel_indices = random.sample(range(len(df_raw)), num_pixels)



        self.df = df_raw.iloc[pixel_indices]
        self.pix_ids = self.df.pixel_id.to_numpy()

        self.initialise_inputs()

        self.initialise_lengths()

        # Target
        self.label = np.random.rand(self.num_pixels*self.max_ccds)


        # Mask Variable Len Sets
        #self.set_max_set_len()

    def set_targets(self, gal_type):
        # Features and inputs:
        self.target = None
        self.target = self.df[gal_type].to_numpy()

    def initialise_lengths(self):
        self.lengths = np.zeros(self.num_pixels, dtype=int)
        if self.var_set_len:
            for i, pix in enumerate(self.pix_ids):
                c = len(self.pixel2ccd_dict[pix])
                if c < self.max_ccds:
                    self.lengths[i] = c
                else:
                    self.lengths[i] = self.max_ccds

        else:
            self.lengths.fill(self.max_ccds)

    def initialise_inputs(self):
        #self.input = -1 * np.ones((self.num_pixels, self.max_ccds, self.num_features))
        self.input = np.zeros((self.num_pixels, self.max_ccds, self.num_features))

        # Iterate through the pixels
        for i, pix in enumerate(self.pix_ids):
            ids = self.pixel2ccd_dict[pix]
            random.shuffle(ids)
            #print(len(ids))
            ids = ids[:30]
            #print(len(ids))
            #print()
            x = self.ccd.get_ccds(ids)
            # Iterate through the CCDs for every pixel
            for j in range(len(ids)):
                self.input[i, j] = x[j]

    def set_max_set_len(self):
        self.index_matrix =  -1*np.ones((self.num_pixels, self.max_ccds), dtype=int)

        # Getting random labels for now, in the future this will be the output densities

        m = 0
        for i in range(self.num_pixels):

            for j in range(self.lengths[i]):
                ''' This code with label == 0 is not yet needed, but this masking will become necessary when I have
                    I have 64 subpixels per pixel and some of those are not covered by CCDs'''
                while self.label[m] == 0:
                    m += 1
                self.index_matrix[i, j] = m
                m += 1

        print(self.lengths)
        print(self.index_matrix)

    def __len__(self):
        return self.num_pixels

    def __getitem__(self, idx):
        x = torch.from_numpy(self.input[idx]).float()
        #x = x.unsqueeze(0)
        y = torch.tensor(self.target[idx]).float()
        y = y.unsqueeze(-1)
        #l = torch.tensor(self.lengths[idx])
        l = self.lengths[idx]

        return x,y,l


""" Todo
1. Where to get the data from
2. Scaling --> import an already scaled dataset, this will have to be prepared but should be same for Neural Net
3. Combine larger and smaller dataset
4. Build 64 input channels instead of one, so one more dimension of tensors( NO of Pixels,no_of_subpixels,no_ccds, no_features)
"""

' Todo\n1. Where to get the data from\n2. Scaling --> import an already scaled dataset, this will have to be prepared but should be same for Neural Net\n3. Combine larger and smaller dataset\n4. Build 64 input channels instead of one, so one more dimension of tensors( NO of Pixels,no_of_subpixels,no_ccds, no_features)\n'

[172659, 66312, 200133, 14751, 172602, 110234, 252897, 66256, 212584, 241527]
13
13

14
14

13
13

13
13

6
6

14
14

28
28

10
10

31
30

27
27

[13 14 13 13  6 14 28 10 30 27]


## Building the Actual Network Architecture



In [5]:
r"""
Permutation Equivariant and Permutation Invariant layers, as described in the
paper Deep Sets, by Zaheer et al. (https://arxiv.org/abs/1703.06114)
"""

import math

import torch
from torch import nn
from torch.nn import init


class InvLinear(nn.Module):
    r"""Permutation invariant linear layer.
    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        bias: If set to False, the layer will not learn an additive bias.
            Default: ``True``
        reduction: Permutation invariant operation that maps the input set into a single
            vector. Currently, the following are supported: mean, sum, max and min.
    """
    def __init__(self, in_features, out_features, bias=True, reduction='mean'):
        super(InvLinear, self).__init__()

        self.in_features = in_features
        self.out_features = out_features
        assert reduction in ['mean', 'sum', 'max', 'min'],  \
            '\'reduction\' should be \'mean\'/\'sum\'\'max\'/\'min\', got {}'.format(reduction)
        self.reduction = reduction

        self.beta = nn.Parameter(torch.Tensor(self.in_features,
                                              self.out_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(1, self.out_features))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        init.xavier_uniform_(self.beta)
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.beta)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)

    def forward(self, X, mask=None):
        r"""
        Maps the input set X = {x_1, ..., x_M} to a vector y of dimension out_features,
        through a permutation invariant linear transformation of the form:
            $y = \beta reduction(X) + bias$
        Inputs:
        X: N sets of size at most M where each element has dimension in_features
           (tensor with shape (N, M, in_features))
        mask: binary mask to indicate which elements in X are valid (byte tensor
            with shape (N, M) or None); if None, all sets have the maximum size M.
            Default: ``None``.
        Outputs:
        Y: N vectors of dimension out_features (tensor with shape (N, out_features))
        """
        #print("INVLAYER:", X.shape)
        N, M, _ = X.shape
        device = X.device
        y = torch.zeros(N, self.out_features).to(device)
        if mask is None:
            mask = torch.ones(N, M).byte().to(device)

        if self.reduction == 'mean':
            sizes = mask.float().sum(dim=1).unsqueeze(1)
            Z = X * mask.unsqueeze(2).float()
            y = (Z.sum(dim=1) @ self.beta)/sizes

        elif self.reduction == 'sum':
            Z = X * mask.unsqueeze(2).float()
            y = Z.sum(dim=1) @ self.beta

        elif self.reduction == 'max':
            Z = X.clone()
            Z[~mask] = float('-Inf')
            y = Z.max(dim=1)[0] @ self.beta

        else:  # min
            Z = X.clone()
            Z[~mask] = float('Inf')
            y = Z.min(dim=1)[0] @ self.beta

        if self.bias is not None:
            y += self.bias

        return y

    def extra_repr(self):
        return 'in_features={}, out_features={}, bias={}, reduction={}'.format(
            self.in_features, self.out_features,
            self.bias is not None, self.reduction)


class EquivLinear(InvLinear):
    r"""Permutation equivariant linear layer.
    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        bias: If set to False, the layer will not learn an additive bias.
            Default: ``True``
        reduction: Permutation invariant operation that maps the input set into a single
            vector. Currently, the following are supported: mean, sum, max and min.
    """
    def __init__(self, in_features, out_features, bias=True, reduction='mean'):
        super(EquivLinear, self).__init__(in_features, out_features,
                                          bias=bias, reduction=reduction)

        self.alpha = nn.Parameter(torch.Tensor(self.in_features,
                                               self.out_features))

        self.reset_parameters()

    def reset_parameters(self):
        super(EquivLinear, self).reset_parameters()
        if hasattr(self, 'alpha'):
            init.xavier_uniform_(self.alpha)

    def forward(self, X, mask=None):
        r"""
        Maps the input set X = {x_1, ..., x_M} to the output set
        Y = {y_1, ..., y_M} through a permutation equivariant linear transformation
        of the form:
            $y_i = \alpha x_i + \beta reduction(X) + bias$
        Inputs:
        X: N sets of size at most M where each element has dimension in_features
           (tensor with shape (N, M, in_features))
        mask: binary mask to indicate which elements in X are valid (byte tensor
            with shape (N, M) or None); if None, all sets have the maximum size M.
            Default: ``None``.
        Outputs:
        Y: N sets of same cardinality as in X where each element has dimension
           out_features (tensor with shape (N, M, out_features))
        """
        N, M, _ = X.shape
        device = X.device
        Y = torch.zeros(N, M, self.out_features).to(device)
        if mask is None:
            mask = torch.ones(N, M).byte().to(device)

        Y = torch.zeros(N, M, self.out_features).to(device)
        h_inv = super(EquivLinear, self).forward(X, mask=mask)
        Y[mask] = (X @ self.alpha + h_inv.unsqueeze(1))[mask]

        return Y


In [6]:
class SetNet(nn.Module):
    def __init__(self, n_features = 5, n_output = 3, reduction = 'sum'):
        super(SetNet,self).__init__()

        # Takes an Input Tensor and applies transformations to last layer --> features
        # Output of Feature Layer: Tensor with Max.CCDs elements, which can now be passed to Set Layer
        self.feature_extractor = nn.Sequential(
            nn.Linear(n_features,4),
            nn.ReLU(inplace=True),
            nn.Linear(4,3),
            nn.ReLU(inplace=True),
            nn.Linear(3,n_output),
            nn.ReLU(inplace=True)
        )

        self.adder = InvLinear(3,1, reduction=reduction, bias=True)


        # Invariant Layer Influenced by Code from DPernes, but adapted for the current regression task instead of CNN

    def forward(self, X, mask = None):
        y = self.feature_extractor(X)

        y = self.adder(y, mask=mask)
        return y


In [20]:
traindata = SetSequence(var_set_len=True)
traindata.set_targets('lrg')
x,y,l = traindata.__getitem__(3)
print(l)
print(y)
print(y.shape)

30
tensor(47.)
torch.Size([])


In [78]:
net = SetNet()
y = net.forward(x)
print(y)


INVLAYER: torch.Size([1, 30, 3])
tensor([[-8.7350]], grad_fn=<AddBackward0>)


In [80]:
#Work out masking logic
device = 'cpu'

x = x.to(device)
y = y.to(device)
l = l.to(device)
def get_mask(sizes, max_size):

    return (torch.arange(max_size).reshape(1, -1).to(sizes.device) < sizes.reshape(-1, 1))

mask = get_mask(l, x.shape[1])
print(mask)

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False]])


###  Hyperparameters and Training Loops

In [7]:
# Defining Loss
criterion = nn.MSELoss()

#Defining Hyperparemeters
no_epochs = 100 #very low, but computational power not sufficient for more iterations
batch = 4
learning_rate = 0.001

#Using the Adam Method for Stochastic Optimisation
#optimiser = optim.Adam(model.parameters(), lr=learning_rate)

galaxy_types = ['lrg', 'elg', 'qso']
device = 'cpu'
def get_mask(sizes, max_size):
    return (torch.arange(max_size).reshape(1, -1).to(sizes.device) < sizes.reshape(-1, 1))


In [34]:
traindata = SetSequence( num_pixels=1000, var_set_len=True)
for gal in galaxy_types:
    model = SetNet(n_features=traindata.num_features, reduction='sum').to(device)
    optimiser = optim.Adam(model.parameters(), lr=learning_rate)
    print("GALAXY TYPE: ", gal)
    print()
    traindata.set_targets(gal_type=gal)

    time_start = time.time()

    for epoch in range(no_epochs):
        loss_per_epoch = 0
        #loading the training data from trainset and shuffling for each epoch
        trainloader = torch.utils.data.DataLoader(traindata, batch_size=batch, shuffle = True)

        for i, (X, labels, set_sizes) in enumerate(trainloader):
            #Put Model into train mode
            model.train()

            #Extract inputs and associated labels from dataloader batch
            X = X.to(device)

            labels = labels.to(device)
            set_sizes = set_sizes.to(device)

            mask = get_mask(set_sizes, X.shape[1])

            #Predict outputs (forward pass)
            predictions =  model(X, mask=mask)
            #Compute Loss
            loss = criterion(predictions, labels)

            #Zero-out the gradients before backward pass (pytorch stores the gradients)
            optimiser.zero_grad()

            #Backpropagation
            loss.backward()

            #Perform one step of gradient descent
            optimiser.step()

            #Append loss to the general loss for this one epoch
            loss_per_epoch += loss.item()
        if epoch % 10 == 0:
            print("Loss for Epoch", epoch, ": ", loss_per_epoch)
    time_end = time.time()
    time_passed = time_end - time_start
    print()
    print(f"{time_passed/60:.5} minutes ({time_passed:.3} seconds) taken to train the model")

GALAXY TYPE:  lrg

Loss for Epoch 0 :  98861.45623016357
Loss for Epoch 10 :  52696.973711013794
Loss for Epoch 20 :  50914.9296233654
Loss for Epoch 30 :  49587.04089486599
Loss for Epoch 40 :  48442.36259889603
Loss for Epoch 50 :  47446.274691581726
Loss for Epoch 60 :  46596.20416927338
Loss for Epoch 70 :  45834.78798055649
Loss for Epoch 80 :  45298.984749794006
Loss for Epoch 90 :  44603.317781448364

0.31246 minutes (18.7 seconds) taken to train the model
GALAXY TYPE:  elg

Loss for Epoch 0 :  2523104.477661133
Loss for Epoch 10 :  1067209.6954803467
Loss for Epoch 20 :  1008137.7860870361
Loss for Epoch 30 :  951869.201921463
Loss for Epoch 40 :  898654.8865890503
Loss for Epoch 50 :  848450.6083221436
Loss for Epoch 60 :  801020.8725280762
Loss for Epoch 70 :  756657.4473953247
Loss for Epoch 80 :  715235.8232269287
Loss for Epoch 90 :  676599.9020805359

0.30592 minutes (18.4 seconds) taken to train the model
GALAXY TYPE:  qso

Loss for Epoch 0 :  1087824.825958252
Loss for 